In [15]:
import logging
from logging.handlers import RotatingFileHandler

import aiohttp
import asyncio
import requests, json

#### SIMPLE POST REQUEST

In [3]:
api_url = 'http://192.168.18.17:3021/datalogger/create/'

In [4]:
# todos = {'userId': 1, 'title': 'Buy milk', 'completed': False}
with open('./saved_data/20240806105406_WONDFOBGA_0001_ASROFI.txt', 'r') as f:
    send_data = f.read()

In [5]:
dict_data = json.loads(send_data)

In [27]:
response = requests.post(api_url, json=dict_data)

In [28]:
response.status_code

201

In [30]:
response.json()

{'id': 201,
 'merk': 'BGA-101',
 'date': '2024-06-13 13:28:21',
 'parameter': [{'parameter': 'BodyTemperature',
   'nilai': '37.0',
   'success': 'true',
   'note': '',
   'image': '-'},
  {'parameter': 'FIO2',
   'nilai': '0.45',
   'success': 'true',
   'note': '',
   'image': '-'},
  {'parameter': 'pO2(A-a)(T)',
   'nilai': '106',
   'success': 'true',
   'note': '',
   'image': '-'},
  {'parameter': 'pO2(A-a)',
   'nilai': '106',
   'success': 'true',
   'note': '',
   'image': '-'},
  {'parameter': 'pO2(a/A)(T)',
   'nilai': '0.54',
   'success': 'true',
   'note': '',
   'image': '-'},
  {'parameter': 'pO2(a/A)',
   'nilai': '0.54',
   'success': 'true',
   'note': '',
   'image': '-'},
  {'parameter': 'pO2(T)/FIO2',
   'nilai': '282',
   'success': 'true',
   'note': '',
   'image': '-'},
  {'parameter': 'pO2/FIO2',
   'nilai': '282',
   'success': 'true',
   'note': '',
   'image': '-'},
  {'parameter': 'RI(T)',
   'nilai': '0.84',
   'success': 'true',
   'note': '',
   'image

#### HANDLE RESPONSE AND ERROR

In [ ]:
if response.status_code == 200:
    print('Data sent successfully')
elif response.status_code == 201:
    print('Data created successfully')
else:
    print('Error:', response.status_code)

In [ ]:
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as err:
    print(f'HTTP error: {err}')
except Exception as err:
    print(f'Other error: {err}')

#### USING SESSION

In [7]:
session = requests.Session()
session.headers.update({'Content-Type': 'application/json'})

response = session.post(api_url, json=dict_data)
print(f'Response status code: {response.status_code}')

C:\Users\nforc\AppData\Local\Programs\Python\Python39\lib\ast.py:50: RuntimeWarning: coroutine 'send_data' was never awaited
  return compile(source, filename, mode, flags,


ConnectionError: HTTPConnectionPool(host='192.168.18.17', port=3021): Max retries exceeded with url: /datalogger/create/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000027E392E8FA0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

#### USING ASYNCIO AND AIOHTTP

In [6]:
async def send_data(url, data):
    async with aiohttp.ClientSession() as session:
        async with session.post(url, json=data, timeout=10) as session:
            print(f'Response status code: {response.status}')
            print(f'Response text: {await response.text()}')
            
asyncio.run(send_data(url=api_url, data=dict_data))

RuntimeError: asyncio.run() cannot be called from a running event loop

#### LOGGING

In [8]:
logging.basicConfig(level=logging.INFO)
logging.info('Data sent successfully')
logging.error('Failed to send data')

INFO:root:Data sent successfully
ERROR:root:Failed to send data


#### SEND DATA WITH LOGGING

In [11]:
logging.basicConfig(
    filename='send_data.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [16]:
file_handler = RotatingFileHandler('send_data.log', maxBytes=2000, backupCount=10)
stream_handler = logging.StreamHandler()
logging.basicConfig(
    handlers=[file_handler, stream_handler],
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [17]:
def send_data_with_logging(url, data):
    try:
        response = requests.post(url, json=data)
        response.raise_for_status()
        logging.info('Data sent successfully')
    except requests.exceptions.HTTPError as err:
        logging.error(f'HTTP error: {err}')
    except Exception as err:
        logging.error(f'Other error: {err}')

In [18]:
send_data_with_logging(api_url, dict_data)

ERROR:root:Other error: HTTPConnectionPool(host='192.168.18.17', port=3021): Max retries exceeded with url: /datalogger/create/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000027E39E07850>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))
